### 아이디어
- 예제 입력을 비교해봤을 때, $b_i$ 가 낮을 수록 수열을 구성하기 쉽다고 볼 수 있다.
- (관찰) $b_i$ 가 $\text{MEX}(A[:i])$ 라는 의미는 $i$ 이전에 $b_i$ 보다 낮은 숫자들이 한 칸을 제외하고 모두 채워져 있다가, \
오직 $b_i$ 에서 그 한 칸이 채워졌다는 의미이다.
  - $b_i > i$ 라면 숫자를 빈 칸 없이 채우는 방법이 없으므로 성립하지 않게 된다.
  - 마찬가지로 $B$ 의 마지막 숫자는 정확히 $N + 1$ 이여야만 성립한다.
  - 구간 $[1, b_i]$ 에서 $b_i - 1$ 이 upper bound인 숫자가 정확히 $b_i - 1$ 번 나와야 한다.
  - 추가로, $a_i < b_i$ 가 성립해야 한다. $b_i$ 보다 높다면, MEX 값이 $i$ 에서 늘어날 수가 없다. \
  그렇다면 $a_i < b_i$ 인 원소 1개가 $i$에 배치되었으므로, 구간 $[1, b_i)$ 에는 $b_i$ 보다 낮은 숫자가 $b_i - 2$ 번 나와야 할 것이다.
  - 이를 수직선 상에서 이해한다면, 더 왼쪽에 있는 원소에 대해서 더 많은 제약이 가해지는 것으로 볼 수 있다. \
  수열을 거꾸로 채워나간다면 Greedy하게 채워나갈 수 있을 것이다.
- 자연스럽게 exchange argument가 된 것 같다. 아마도 이 문제는 그리디 문제일 것이다.

- 구현을 하려고 보니까 argument가 하나 더 생긴 것 같다. (미리 따질 수 있었을까?) 
  - $B$ 에서 증가하는 점. 즉, $b_i < b_{i+1}$ 를 만족하는 $b_{i+1}$ 를 $b_j$ 라고 하자.\
  $a_{j+1} = b_j$ 일 수 밖에 없다. 왜냐하면, 그 사이 구간에서 $b_j$ 가 나왔다면, MEX가 1 증가해야 하기 때문이다.
  - 여전히 뒤에서 부터 수열을 채우는 greedy는 여전히 valid 하다. 단지 $a_j$ 에 대한 argument가 하나 더 추가된 것 뿐이기 때문이다. \
  $B$ 가 정렬된 수열이라는 점을 이용해 이분 탐색으로 이전의 증가하는 점을 $O(\log N)$ 에 찾을 수 있을 것이다.
    - 이 경우 $L$ 을 순회할 때, bisect_left($b_j$ + 1) 에 $b_j$ 를 배치하면 될 것 같다.

In [ ]:
import io, os, sys, bisect, heapq
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def sol() :
  N = int(input())
  L = list(map(int, input().split()))
  if L[-1] != N + 1: return
  res = [-1] * N
  
  lo = 0 # 마지막으로 증가한 점의 값
  hi = N #현재 구간에서 upper bound 구간 
  cnt = 0 #(lo, hi)에 제약되는 원소의 개수
  NUM = {*range(1, N+1)}
  for i, v in enumerate(L[:-1], 1) :
    if v > i + 1 : return
    if v > lo : # 증가한 점이 있으면
      lo = v
      pt = bisect.bisect_left(L, lo + 1)
      hi = L[pt]
      res[pt] = v
      NUM.remove(v)
      cnt += hi - lo - 1
    elif cnt :
      NUM.remove(lo + cnt) #반드시 성립해야
      res[i-1] = lo + cnt
      cnt -= 1
  
  for i, v in enumerate(res) :
    if v == -1 :
      res[i] = NUM.pop()
  
  return res

if ans := sol() :
  sys.stdout.write("YES\n" + " ".join(map(str, ans)))
else :
  sys.stdout.write("NO")

- WA. 중간에 cnt가 0이 아닌 상태에서 또 늘어난다면 중복되는 원소가 빠져나올 수 있다.

In [ ]:
import io, os, sys, bisect, heapq
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def sol() :
  N = int(input())
  L = list(map(int, input().split()))
  if L[-1] != N + 1: return
  if L != sorted(L) : return
  res = [-1] * N
  
  lo = 0 # 마지막으로 증가한 점의 값
  hi = N #현재 구간에서 upper bound 구간 
  cnt = 0 #(lo, hi)에 제약되는 원소의 개수
  NUM = {*range(1, N+1)}
  for i, v in enumerate(L[:-1], 1) :
    if v > i + 1 : return

    if v > lo : # 증가한 점이 있으면
      lo = v
      pt = bisect.bisect_left(L, lo + 1)
      hi = L[pt]
      res[pt] = v
      NUM.remove(v)
      cnt += hi - lo - 1

    if cnt and res[i-1] == -1 :
      NUM.remove(lo + cnt)
      res[i-1] = lo + cnt
      cnt -= 1
  
  if NUM :
    for i, v in enumerate(res) :
      if v == -1 :
        res[i] = NUM.pop()
  
  return res

if ans := sol() :
  sys.stdout.write("YES\n" + " ".join(map(str, ans)))
else :
  sys.stdout.write("NO")

- 반례: 2 2 3 5 5 7 / 2 2 4

### 다른 접근
- 아이디어는 맞는 것 같은데 구현이 심각하게 꼬였다. 기존에 있던 관찰을 다시볼 필요가 있다.
- $a_{j+1} = b$ 인 점들을 모두 확정하고 나면, 모든 수들이 가지는 남은 조건들을 다시 따져보자
  - $a_i \ne b_i$: MEX 이니 자명하다.
  - $a_i < b_{j+1} < b_{j+2} < \cdots$: 잘 생각해보면 왼쪽에 있는 원소일수록 가능한 범위가 낮아진다
  - 어짜피 MEX에 영향을 주는 숫자들을 미리 제거했으니, 위치들을 확정하고 나면 그냥 남은 숫자들을 가능한 낮은 순서대로 채워넣어도 성립하지 않을까?

In [ ]:
import io, os, sys, bisect, heapq
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def sol() :
  N = int(input())
  L = list(map(int, input().split()))
  if L[-1] != N + 1: return
  if L != sorted(L) : return #O(N)도 가능
  res = [-1] * N
  
  lo = 0 # 마지막으로 증가한 점의 값
  NUM = {*range(1, N+1)}
  if L[0] == 2 :
    res[0] = 1
    lo = 2
    NUM.remove(1)

  for i, v in enumerate(L[:-1], 1) :
    if v > i + 1 : return
    if v > lo : # 증가한 점이 있으면
      lo = v
      pt = bisect.bisect_left(L, lo + 1)
      assert res[pt] == -1 #애초에 그런 점이 있으면 서로 다른 값이였을 것이다.
      NUM.remove(v)
      res[pt] = v

  NUM = list(NUM)
  heapq.heapify(NUM)
  for i, v in enumerate(res) :
    if v == -1 :
      x = heapq.heappop(NUM)
      if x == L[i] : 
        tmp, x = x, heapq.heappop(NUM)
        heapq.heappush(NUM, tmp)
      res[i] = x
  
  return res

if ans := sol() :
  sys.stdout.write("YES\n" + " ".join(map(str, ans)))
else :
  sys.stdout.write("NO")

- WA 받았다. 더 이상 틀린곳을 구할수도 없는데..

In [ ]:
import io, os, sys, bisect, heapq
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

def sol() :
  N = int(input())
  L = list(map(int, input().split()))
  if L[-1] != N + 1: return
  if L != sorted(L) : return #O(N)도 가능
  res = [-1] * N
  
  lo = 0 # 마지막으로 증가한 점의 값
  NUM = {*range(1, N+1)}

  if L[0] == 2 :
    res[0] = 1
    lo = 2
    NUM.remove(1)
  for i, v in enumerate(L[:-1], 1) :
    if v > i + 1 : return
    if v > lo : # 증가한 점이 있으면
      lo = v
      pt = bisect.bisect_left(L, lo + 1)
      assert res[pt] == -1 #애초에 그런 점이 있으면 서로 다른 값이였을 것이다.
      NUM.remove(v)
      res[pt] = v

  NUM = list(NUM)
  heapq.heapify(NUM)
  for i, v in enumerate(res) :
    if v == -1 :
      x = heapq.heappop(NUM)
      if x == L[i] : 
        tmp, x = x, heapq.heappop(NUM)
        heapq.heappush(NUM, tmp)
      res[i] = x
  
  return res

if ans := sol() :
  sys.stdout.write("Yes\n" + " ".join(map(str, ans)))
else :
  sys.stdout.write("No")

### 자력솔
- 아 씨발 대소문자
- 그리디인 것을 알았다면, 부분적인 관찰이 끝났다면, 좀 더 넓은 범위에서 관찰해보는 것이 좋을 것 같다.